In [1]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [2]:
df = pd.read_csv('data1/서울_주유소_230418.csv')
df.head()

,상호,주소,상표,가격,경유,셀프,구
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1623,1565,Y,강남구
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,1669,1575,Y,강남구
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),현대오일뱅크,1684,1609,Y,강남구
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,1687,1639,Y,강남구
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,1704,1669,Y,강남구


### 구별로 집계 분석

구 / mean / max / min

In [3]:
df.drop(['경유'], axis = 1, inplace = True)

In [4]:
df.groupby('구')['가격'].mean()

구
강남구     1942.294118
강동구     1735.000000
강북구     1649.916667
강서구     1696.818182
관악구     1744.714286
광진구     1684.538462
구로구     1737.200000
금천구     1699.090909
노원구     1699.428571
도봉구     1673.687500
동대문구    1687.055556
동작구     1738.333333
마포구     1699.909091
서대문구    1669.428571
서초구     1737.033333
성동구     1797.062500
성북구     1693.782609
송파구     1684.827586
양천구     1676.291667
영등포구    1731.851852
용산구     2169.500000
은평구     1639.000000
종로구     1951.000000
중구      2053.400000
중랑구     1661.153846
Name: 가격, dtype: float64

In [5]:
gu_mm = df.groupby('구')['가격'].agg(['mean','min','max'])
gu_mm

,mean,min,max
구,,,
강남구,1942.294118,1623,2578
강동구,1735.000000,1669,1978
강북구,1649.916667,1629,1707
강서구,1696.818182,1628,1995
관악구,1744.714286,1685,1852
광진구,1684.538462,1654,1748
구로구,1737.200000,1645,2198
금천구,1699.090909,1665,1769
노원구,1699.428571,1629,1795


In [6]:
# 해당 구에서 최대값, 최소값만 구하기
_, mi, ma = gu_mm.loc['강남구']   # '_' 는 익명처리한 것임. 없으면 에러남

In [7]:
mi, ma = int(mi), int(ma)
mi, ma

(1623, 2578)

In [8]:
# 컬럼 추가, 최대값인지 최소값인지 결정에서 넣음
df['mm'] = np.zeros(df.shape[0], int)
df.head()

,상호,주소,상표,가격,셀프,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1623,Y,강남구,0
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,1669,Y,강남구,0
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),현대오일뱅크,1684,Y,강남구,0
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,1687,Y,강남구,0
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,1704,Y,강남구,0


In [9]:
df.shape[0]

441

In [13]:
# mm컬럼에 최소값이 같으면 1을, 최대값과 같으면 2를 삽입
gu_mm.loc['강남구']

mean    1942.294118
min     1623.000000
max     2578.000000
Name: 강남구, dtype: float64

In [14]:
gu_mm.index

Index(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구',
       '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구',
       '용산구', '은평구', '종로구', '중구', '중랑구'],
      dtype='object', name='구')

In [15]:
df.index

RangeIndex(start=0, stop=441, step=1)

In [17]:
df.구[1]

'강남구'

In [21]:
for gu in gu_mm.index:
    _, mi, ma = gu_mm.loc[gu]
    mi, ma = int(mi), int(ma)
    for i in df.index:
        if df.가격 [i] == mi and df.구[i] == gu:
            df.mm[i] = 1
        if df.가격 [i] == ma and df.구[i] == gu:
            df.mm[i] = 2

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_3864\1888534535.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 1
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_3864\1888534535.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.mm[i] = 2


In [24]:
df.head()

,상호,주소,상표,가격,셀프,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1623,Y,강남구,1
1,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,1669,Y,강남구,0
2,삼성동주유소,서울 강남구 테헤란로 619 (삼성동),현대오일뱅크,1684,Y,강남구,0
3,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,1687,Y,강남구,0
4,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,1704,Y,강남구,0


In [25]:
# mm컬럼에 데이터가 있는 행만 추출
df1 = df[df.mm !=0]
df1.head()         

,상호,주소,상표,가격,셀프,구,mm
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1623,Y,강남구,1
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),GS칼텍스,2578,N,강남구,2
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),현대오일뱅크,1669,Y,강동구,1
46,광성주유소,서울 강동구 올림픽로 673 (천호동),S-OIL,1978,N,강동구,2
47,(주)자연에너지 국민주유소,서울 동대문구 답십리로 223 (답십리동),현대오일뱅크,1615,Y,동대문구,1


In [26]:
df1.shape

(70, 7)

## 지도시각화

In [29]:
# KAKAO API로 위도 경도값을 추출
with open('./kakao_API.txt') as file:
    kakao_key = file.read()

In [31]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(df1.주소[i])}'    #query를 보낼 때는 ?로 연결됨
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%20%EB%8F%84%EB%B4%89%EA%B5%AC%20%EB%B0%A9%ED%95%99%EB%A1%9C%20151'

In [40]:
# 도로명주소 옆에 위도, 경도 값 추가

lat_list = []
lng_list = []
for i in df1.index:
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(df1.주소[i])}'  
    result = requests.get(url, headers = {"Authorization": f'KakaoAK {kakao_key}'}).json()
    try:
        lng_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:   # 행정동명으로 표기되어 있지 않아서 못 받아오는 경우
        print(df1.상호[i])

In [41]:
lng_list, lat_list

([127.118742598267,
  127.047114626083,
  127.149372429092,
  127.124241977598,
  127.064971211446,
  127.057395478312,
  126.961445941667,
  126.922881347334,
  126.951389038225,
  126.917179478209,
  126.912557082288,
  126.948522081226,
  126.943337675172,
  126.936581879011,
  126.932458194132,
  127.042624862658,
  127.042231529687,
  127.036537420429,
  127.027391824017,
  127.010120328789,
  127.00997322889,
  127.029473489653,
  127.132288546135,
  127.106656298253,
  127.12364117535,
  127.118061832225,
  126.82701689593,
  126.829178494291,
  126.833868932502,
  126.908251942382,
  126.899464416873,
  126.902813303561,
  126.895817488746,
  126.908792674047,
  126.925570104479,
  127.019086440546,
  127.032887089001,
  127.002896422503,
  126.968993700509,
  126.902034060396,
  126.9242140035,
  126.985811114051,
  126.966616657335,
  127.015691795633,
  126.972171955834,
  127.080281126703,
  127.077627212682,
  126.8414715329,
  126.846964931559,
  126.862833597877,
  126.8

In [42]:
df1['위도'] = lat_list
df1['경도'] = lng_list

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_3864\197933941.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['위도'] = lat_list
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_3864\197933941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['경도'] = lng_list


In [43]:
df1.head()

,상호,주소,상표,가격,셀프,구,mm,위도,경도
0,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,1623,Y,강남구,1,37.467272,127.118743
33,(주)만정에너지 삼보주유소,서울 강남구 봉은사로 433 (삼성동),GS칼텍스,2578,N,강남구,2,37.511551,127.047115
34,재건에너지 재정제2주유소 고속셀프지점,서울특별시 강동구 천호대로 1246 (둔촌제2동),현대오일뱅크,1669,Y,강동구,1,37.536417,127.149372
46,광성주유소,서울 강동구 올림픽로 673 (천호동),S-OIL,1978,N,강동구,2,37.541577,127.124242
47,(주)자연에너지 국민주유소,서울 동대문구 답십리로 223 (답십리동),현대오일뱅크,1615,Y,동대문구,1,37.573012,127.064971


In [50]:
# 마커표시

map = folium.Map(location = [37.5502, 126.982], zoom_start = 12)
for i in df1.index:
    folium.Marker(location = [df1.위도[i], df1.경도[i]],
                 popup = folium.Popup(df.주소[i], max_width = 200),
                 tooltip = f'{df.상호[i]}<br>휘발유:{df.가격[i]:,d}원',  # 클릭하면 상호, 가격 출력
                 icon = folium.Icon(color = 'green' if df.mm[i] == 1 else 'red', 
                                   icon = 'glyphicon glyphicon-thumbs-up' if df.mm[i] == 1 else 'glyphicon glyphicon-thumbs-down')
                 ).add_to(map)
map

In [51]:
# 단계구분도 그려보기
filename = './Data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding ='utf-8'))
geo_str

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '강동구',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.16683184366129, 37.57672487388627],
      [127.18408792330152, 37.55814280369575],
      [127.16530984307447, 37.54221851258693],
      [127.14672806823502, 37.51415680680291],
      [127.12123165719615, 37.52528270089],
      [127.1116764203608, 37.540669955324965],
      [127.11519584981606, 37.557533180704915]]]}},
  {'type': 'Feature',
   'id': '송파구',
   'properties': {'code': '11240',
    'name': '송파구',
    'name_eng': 'Songpa-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.0690698130372, 37.522279423505026],
      [127.10087519791962, 37.524841220167055],
      [127.1116764203608, 37.540669955324965],
      [127.12123165719615, 37.52528270089

In [ ]:
# 지도불러오기
map2 = folium.Map(location = [37.581, 126.986], zoom_start = 11)

# 단계구분도
folium.Choropleth(geo_data = geo_str,       # 지도데이터 파일
                  data = df,                 # 시각화하려는 데이터프레임
                  columns = ('동','인구'),   # 동별로 데이터가 있음, 지도데이터와 mapping할 값, 시각화하려는 변수
                  key_on = 'feature.properties.동',    # json파일의 feature? 데이터파일과 매칭할 값
                  fill_color = 'BuPu',
                  legend_name = '휘발유 평균값'   #color의 범주
                 ).add_to(map)

map2